# Mean Reversion Strategies**QuantLearn Module**: Strategy Types**Difficulty**: Intermediate**Time**: ~25 minutesLearn to build strategies that profit when prices return to their mean - the opposite of trend following.

In [ ]:
#@title 📦 Setupimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltfrom scipy import statsnp.random.seed(42)plt.style.use('seaborn-v0_8-whitegrid')# Generate mean-reverting price data (Ornstein-Uhlenbeck process)def generate_mean_reverting_data(n_days=500, mean=100, theta=0.1, sigma=2):    prices = [mean]    for _ in range(n_days - 1):        dp = theta * (mean - prices[-1]) + sigma * np.random.randn()        prices.append(prices[-1] + dp)    dates = pd.date_range('2022-01-01', periods=n_days, freq='B')    prices = np.array(prices)    returns = np.diff(prices) / prices[:-1]    return pd.DataFrame({        'Price': prices,        'Return': [np.nan] + list(returns)    }, index=dates)df = generate_mean_reverting_data()print("✓ Setup complete!")

## 1. Bollinger BandsTrade when price deviates significantly from its moving average:- **Upper Band** = MA + 2σ- **Lower Band** = MA - 2σ- **Signal**: Buy at lower band, sell at upper band

In [ ]:
# Bollinger Bands Strategywindow = 20num_std = 2df['MA'] = df['Price'].rolling(window).mean()df['Std'] = df['Price'].rolling(window).std()df['Upper'] = df['MA'] + num_std * df['Std']df['Lower'] = df['MA'] - num_std * df['Std']# Z-score: how many std devs from meandf['Z_Score'] = (df['Price'] - df['MA']) / df['Std']# Signal: buy when below -2, sell when above +2df['BB_Signal'] = 0df.loc[df['Z_Score'] < -num_std, 'BB_Signal'] = 1   # Oversold -> buydf.loc[df['Z_Score'] > num_std, 'BB_Signal'] = -1   # Overbought -> sell# Hold position until opposite signaldf['BB_Signal'] = df['BB_Signal'].replace(0, np.nan).ffill().fillna(0)# Calculate returnsdf['BB_Position'] = df['BB_Signal'].shift(1)df['BB_Return'] = df['BB_Position'] * df['Return']# Visualizefig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)axes[0].plot(df['Price'], label='Price', alpha=0.8)axes[0].plot(df['MA'], label='20-day MA', linewidth=2)axes[0].fill_between(df.index, df['Lower'], df['Upper'], alpha=0.2, label='Bollinger Bands')axes[0].legend()axes[0].set_title('Bollinger Bands Mean Reversion')axes[1].plot(df['Z_Score'], label='Z-Score')axes[1].axhline(2, color='red', linestyle='--', alpha=0.5)axes[1].axhline(-2, color='green', linestyle='--', alpha=0.5)axes[1].axhline(0, color='gray', linestyle='-', alpha=0.3)axes[1].set_ylabel('Z-Score')axes[1].legend()plt.tight_layout()plt.show()

## 2. RSI (Relative Strength Index)Momentum oscillator that measures overbought/oversold conditions:$$RSI = 100 - \frac{100}{1 + RS}$$Where RS = Average Gain / Average Loss over N periods- **RSI > 70**: Overbought → Sell signal- **RSI < 30**: Oversold → Buy signal

In [ ]:
# RSI Strategydef calculate_rsi(prices, period=14):    delta = prices.diff()    gain = delta.where(delta > 0, 0)    loss = (-delta).where(delta < 0, 0)    avg_gain = gain.rolling(period).mean()    avg_loss = loss.rolling(period).mean()    rs = avg_gain / avg_loss    rsi = 100 - (100 / (1 + rs))    return rsidf['RSI'] = calculate_rsi(df['Price'], period=14)# Signal: buy when oversold, sell when overboughtdf['RSI_Signal'] = 0df.loc[df['RSI'] < 30, 'RSI_Signal'] = 1   # Oversold -> buydf.loc[df['RSI'] > 70, 'RSI_Signal'] = -1  # Overbought -> selldf['RSI_Signal'] = df['RSI_Signal'].replace(0, np.nan).ffill().fillna(0)df['RSI_Position'] = df['RSI_Signal'].shift(1)df['RSI_Return'] = df['RSI_Position'] * df['Return']# Plot RSIfig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)axes[0].plot(df['Price'])axes[0].set_ylabel('Price')axes[0].set_title('Price with RSI Signals')axes[1].plot(df['RSI'], label='RSI')axes[1].axhline(70, color='red', linestyle='--', label='Overbought (70)')axes[1].axhline(30, color='green', linestyle='--', label='Oversold (30)')axes[1].fill_between(df.index, 30, 70, alpha=0.1)axes[1].set_ylabel('RSI')axes[1].set_ylim(0, 100)axes[1].legend()plt.tight_layout()plt.show()

## Strategy Comparison

In [ ]:
# Compare strategiesdf_clean = df.dropna()fig, ax = plt.subplots(figsize=(14, 6))strategies = {    'Buy & Hold': (1 + df_clean['Return']).cumprod(),    'Bollinger Bands': (1 + df_clean['BB_Return']).cumprod(),    'RSI': (1 + df_clean['RSI_Return']).cumprod()}for name, cum_ret in strategies.items():    ax.plot(cum_ret, label=name, linewidth=2 if name != 'Buy & Hold' else 1)ax.set_ylabel('Cumulative Return')ax.set_title('Mean Reversion Strategy Comparison')ax.legend()plt.show()# Print statsprint("\nPerformance Metrics:")print("-" * 50)for name, strategy in [('Bollinger', 'BB_Return'), ('RSI', 'RSI_Return')]:    rets = df_clean[strategy]    print(f"\n{name}:")    print(f"  Annual Return: {rets.mean() * 252 * 100:.1f}%")    print(f"  Annual Vol: {rets.std() * np.sqrt(252) * 100:.1f}%")    print(f"  Sharpe: {rets.mean() / rets.std() * np.sqrt(252):.2f}")

## Exercise: Z-Score Mean ReversionBuild a simple z-score mean reversion strategy:1. Calculate the z-score of price vs 20-day MA2. Enter long when z < -1.5, exit when z > 03. Enter short when z > 1.5, exit when z < 0

In [ ]:
# Exercise: Z-Score strategy with exit rules# TODO: Calculate z-scorez_score = None  # Your code# TODO: Create signals with entry/exit logic# This is trickier - you need to track the current positiondf['ZS_Signal'] = 0  # Your code# TODO: Calculate returnsdf['ZS_Position'] = Nonedf['ZS_Return'] = None

In [ ]:
#@title 💡 Solution# Calculate z-scorez_score = (df['Price'] - df['Price'].rolling(20).mean()) / df['Price'].rolling(20).std()# Entry and exit logicposition = 0positions = []for z in z_score:    if np.isnan(z):        positions.append(0)        continue    # Entry signals    if z < -1.5 and position == 0:        position = 1  # Enter long    elif z > 1.5 and position == 0:        position = -1  # Enter short    # Exit signals    elif position == 1 and z > 0:        position = 0  # Exit long    elif position == -1 and z < 0:        position = 0  # Exit short    positions.append(position)df['ZS_Signal'] = positionsdf['ZS_Position'] = df['ZS_Signal'].shift(1)df['ZS_Return'] = df['ZS_Position'] * df['Return']# Plotdf_zs = df.dropna()plt.figure(figsize=(14, 5))plt.plot((1 + df_zs['Return']).cumprod(), label='Buy & Hold', alpha=0.7)plt.plot((1 + df_zs['BB_Return']).cumprod(), label='Bollinger', alpha=0.7)plt.plot((1 + df_zs['ZS_Return']).cumprod(), label='Z-Score (with exits)', linewidth=2)plt.legend()plt.title('Z-Score Strategy with Entry/Exit Rules')plt.show()print("Z-Score Strategy Stats:")rets = df_zs['ZS_Return']print(f"Annual Return: {rets.mean() * 252 * 100:.1f}%")print(f"Sharpe: {rets.mean() / rets.std() * np.sqrt(252):.2f}")print(f"Time in Market: {(df_zs['ZS_Signal'] != 0).mean() * 100:.0f}%")

## Summary| Strategy | Entry Signal | Exit Signal | Best For ||----------|--------------|-------------|----------|| Bollinger Bands | Price hits band | Opposite band | Range-bound markets || RSI | RSI < 30 or > 70 | RSI crosses 50 | Identifying extremes || Z-Score | |z| > threshold | z crosses zero | Statistical approach |**Key insight**: Mean reversion works when prices oscillate around a mean, but fails spectacularly in trending markets. Always know your market regime!**Next**: Advanced Quant Techniques